# Preparations

### Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from theano import scan
import theano.tensor as T
import pymc3 as pm
import theano
from sklearn.metrics import r2_score
import seaborn as sns
import os, sys, subprocess

### Load Real Data

In [ ]:
data = pd.read_csv(os.path.join('../data/', "2nd POS Modeling Training Data OS2.csv"))

In [ ]:
data['DV'] = ((data['DV'].values - 1) / 2) - 1

observed_R = data.pivot(columns = 'ID', index = 'trialseq', values = 'DV').values[:, np.newaxis, :]

### Occasion Setting Model

##### In its default form, the code block below allows λbar, γ1, and γ2 to scale per their formulas. However, in order to improve α parameter estimation (as described in our report), please set each of these to 1. Please see comments in code below on which code to activate and deactivate by highlighting that code and pressing ctrl / (or, on Mac, command /). 

In [ ]:
def learning_function(stimuli_shown, Λ, λ, prev_V, prev_Vbar, prev_P, prev_N, prev_P2, prev_N2, stimulus_type, OS1_type, OS2_type, αD, αDE, αDEF, αS, αE, αF, αEF, αG, αH, αM, αN, αMN, αU, αUM, αUMN):
    
    Λbar = T.zeros_like(Λ)
    Λbar = T.inc_subtensor(Λbar[0,:], (prev_V[5,:] > 0) * (1 - Λ[0, :])) #Dcs
    Λbar = T.inc_subtensor(Λbar[1,:], (prev_V[3,:] > 0) * (1 - Λ[3, :])) #D1
    Λbar = T.inc_subtensor(Λbar[2,:], (prev_V[5,:] > 0) * (1 - Λ[5, :])) #D2
    Λbar = T.inc_subtensor(Λbar[3,:], (prev_V[3,:] > 0) * (1 - Λ[3, :])) #Ecs
    Λbar = T.inc_subtensor(Λbar[4,:], (prev_V[5,:] > 0) * (1 - Λ[5, :])) #E1
    Λbar = T.inc_subtensor(Λbar[5,:], (prev_V[5,:] > 0) * (1 - Λ[5, :])) #F
    Λbar = T.inc_subtensor(Λbar[6,:], (prev_V[5,:] > 0) * (1 - Λ[6, :])) #S
    Λbar = T.inc_subtensor(Λbar[7,:], (prev_V[7,:] > 0) * (1 - Λ[7, :])) #G
    Λbar = T.inc_subtensor(Λbar[8,:], (prev_V[7,:] > 0) * (1 - Λ[7, :])) #H
    Λbar = T.inc_subtensor(Λbar[9,:], (prev_V[9,:] > 0) * (1 - Λ[9, :])) #Mcs
    Λbar = T.inc_subtensor(Λbar[10,:], (prev_V[11,:] > 0) * (1 - Λ[11, :])) #M1
    Λbar = T.inc_subtensor(Λbar[11,:], (prev_V[11,:] > 0) * (1 - Λ[11, :])) #N
    Λbar = T.inc_subtensor(Λbar[12,:], (prev_V[11,:] > 0) * (1 - Λ[12, :])) #Ucs
    Λbar = T.inc_subtensor(Λbar[13,:], (prev_V[9,:] > 0) * (1 - Λ[9, :])) #U1
    Λbar = T.inc_subtensor(Λbar[14,:], (prev_V[11,:] > 0) * (1 - Λ[11, :])) #U2
    Λbar = T.inc_subtensor(Λbar[15,:], (prev_V[3,:] > 0) * (1 - Λ[3, :])) #D1_abs
    Λbar = T.inc_subtensor(Λbar[16,:], (prev_V[5,:] > 0) * (1 - Λ[5, :])) #D2_abs
    Λbar = T.inc_subtensor(Λbar[17,:], (prev_V[5,:] > 0) * (1 - Λ[5, :])) #E1_abs
    Λbar = T.inc_subtensor(Λbar[18,:], (prev_V[11,:] > 0) * (1 - Λ[11, :])) #M1_abs
    Λbar = T.inc_subtensor(Λbar[19,:], (prev_V[9,:] > 0) * (1 - Λ[10, :])) #U1_abs
    Λbar = T.inc_subtensor(Λbar[20,:], (prev_V[11,:] > 0) * (1 - Λ[11, :])) #U2_abs
    
    #To make λbar = 1, activate "λbar = T.ones_like(Λbar)" code below and 
    #deactivate all "λbar =" code under "Code for λbar Scaling" below.
    
    #Code for λbar = 1
#     λbar = T.ones_like(Λbar)
    
    #Code for λbar Scaling
    λbar = T.zeros_like(Λbar)
    λbar = T.inc_subtensor(λbar[0,:], prev_V[5,:]) #Dcs
    λbar = T.inc_subtensor(λbar[1,:], prev_V[3,:]) #D1
    λbar = T.inc_subtensor(λbar[2,:], prev_V[5,:]) #D2
    λbar = T.inc_subtensor(λbar[3,:], prev_V[5,:]) #Ecs
    λbar = T.inc_subtensor(λbar[4,:], prev_V[5,:]) #E1
    λbar = T.inc_subtensor(λbar[5,:], prev_V[5,:]) #F
    λbar = T.inc_subtensor(λbar[6,:], prev_V[5,:]) #S
    λbar = T.inc_subtensor(λbar[7,:], prev_V[7,:]) #G
    λbar = T.inc_subtensor(λbar[8,:], prev_V[7,:]) #H
    λbar = T.inc_subtensor(λbar[9,:], prev_V[9,:]) #Mcs
    λbar = T.inc_subtensor(λbar[10,:], prev_V[11,:]) #M1
    λbar = T.inc_subtensor(λbar[11,:], prev_V[11,:]) #N
    λbar = T.inc_subtensor(λbar[12,:], prev_V[11,:]) #Ucs
    λbar = T.inc_subtensor(λbar[13,:], prev_V[9,:]) #U1
    λbar = T.inc_subtensor(λbar[14,:], prev_V[11,:]) #U2
    λbar = T.inc_subtensor(λbar[15,:], prev_V[3,:]) #D1_abs
    λbar = T.inc_subtensor(λbar[16,:], prev_V[5,:]) #D2_abs
    λbar = T.inc_subtensor(λbar[17,:], prev_V[5,:]) #E1_abs
    λbar = T.inc_subtensor(λbar[18,:], prev_V[11,:]) #M1_abs
    λbar = T.inc_subtensor(λbar[19,:], prev_V[9,:]) #U1_abs
    λbar = T.inc_subtensor(λbar[20,:], prev_V[11,:]) #U2_abs
    
    
    #Prediction error
    pe_V = λ - prev_V
    pe_Vbar = λbar - prev_Vbar
    pe_P = λ - prev_P
    pe_N = λbar - prev_N
    pe_P2 = λ - prev_P2
    pe_N2 = λbar - prev_N2
    
    #Stimulus-Specific α
    αD = αD*(stimuli_shown[21,:] > 0)
    αDE = αDE*(stimuli_shown[22,:] > 0)
    αDEF = αDEF*(stimuli_shown[23,:] > 0)
    αS = αS*(stimuli_shown[24,:] > 0)
    αE = αE*(stimuli_shown[25,:] > 0)
    αF = αF*(stimuli_shown[26,:] > 0)
    αEF = αEF*(stimuli_shown[27,:] > 0)
    αG = αG*(stimuli_shown[28,:] > 0)
    αH = αH*(stimuli_shown[29,:] > 0)
    αM = αM*(stimuli_shown[30,:] > 0)
    αN = αN*(stimuli_shown[31,:] > 0)
    αMN = αMN*(stimuli_shown[32,:] > 0)
    αU = αU*(stimuli_shown[33,:] > 0)
    αUM = αUM*(stimuli_shown[34,:] > 0)
    αUMN = αUMN*(stimuli_shown[35,:] > 0)

    #To make γ = 1, activate code under "Code for γ = 1" and 
    #deactivate code under "Code for γ Scaling" below.
    
    #Code for γ = 1
#     prev_γ1 = T.zeros_like(prev_V)
#     prev_γ1 = T.set_subtensor(prev_γ1[stimulus_type.nonzero(),:],1)
#     prev_γ2 = T.zeros_like(prev_V)
#     prev_γ2 = T.set_subtensor(prev_γ2[OS1_type.nonzero(),:],1)
    
    #Code for γ Scaling
    prev_γ1 = prev_V * prev_Vbar
    prev_γ2 = prev_P * prev_N

    #Delta formulas
    CS_prev_γ1 = (prev_γ1 * stimuli_shown).sum(axis=0)
    CS_prev_γ2 = (prev_γ2 * stimuli_shown).sum(axis=0)
    ΔV = Λ * (αD + αDE + αDEF + αS + αE + αF + αEF + αG + αH + αM + αN + αMN + αU + αUM + αUMN) * pe_V
    ΔVbar = Λbar * (αD + αDE + αDEF + αS + αE + αF + αEF + αG + αH + αM + αN + αMN + αU + αUM + αUMN) * pe_Vbar
    ΔP = Λ * (αD + αDE + αDEF + αS + αE + αF + αEF + αG + αH + αM + αN + αMN + αU + αUM + αUMN) * CS_prev_γ1 * pe_P
    ΔN = Λbar * (αD + αDE + αDEF + αS + αE + αF + αEF + αG + αH + αM + αN + αMN + αU + αUM + αUMN) * CS_prev_γ1 * pe_N
    ΔP2 = Λ * (αD + αDE + αDEF + αS + αE + αF + αEF + αG + αH + αM + αN + αMN + αU + αUM + αUMN) * CS_prev_γ2 * pe_P2
    ΔN2 = Λbar * (αD + αDE + αDEF + αS + αE + αF + αEF + αG + αH + αM + αN + αMN + αU + αUM + αUMN) * CS_prev_γ2 * pe_N2


    # Only update stimuli that were shown
    ΔV = ΔV * stimuli_shown
    ΔVbar = ΔVbar * stimuli_shown
    ΔP = ΔP * stimuli_shown
    ΔN = ΔN * stimuli_shown
    ΔP2 = ΔP2 * stimuli_shown
    ΔN2 = ΔN2 * stimuli_shown
    
    # Update V, Vbar, P, N, P2, N2
    V = T.zeros_like(prev_V)
    Vbar = T.zeros_like(prev_Vbar)
    P = T.zeros_like(prev_P)
    N = T.zeros_like(prev_N)
    P2 = T.zeros_like(prev_P2)
    N2 = T.zeros_like(prev_N2)
    
    # Only update V and Vbar for CSs. Only update P and N for 1st-order OSs. Only update P2 and N2 for 2nd-order OSs.
    V = T.inc_subtensor(V[T.eq(stimulus_type, 1)], prev_V[T.eq(stimulus_type, 1)] + ΔV[T.eq(stimulus_type, 1)])
    Vbar = T.inc_subtensor(Vbar[T.eq(stimulus_type, 1)], prev_Vbar[T.eq(stimulus_type, 1)] + ΔVbar[T.eq(stimulus_type, 1)])
    P = T.inc_subtensor(P[T.eq(OS1_type, 1)], prev_P[T.eq(OS1_type, 1)] + ΔP[T.eq(OS1_type, 1)])
    N = T.inc_subtensor(N[T.eq(OS1_type, 1)], prev_N[T.eq(OS1_type, 1)] + ΔN[T.eq(OS1_type, 1)])
    P2 = T.inc_subtensor(P2[T.eq(OS2_type, 1)], prev_P2[T.eq(OS2_type, 1)] + ΔP2[T.eq(OS2_type, 1)])
    N2 = T.inc_subtensor(N2[T.eq(OS2_type, 1)], prev_N2[T.eq(OS2_type, 1)] + ΔN2[T.eq(OS2_type, 1)])
    
    return V, Vbar, P, N, P2, N2

### Generate Simulated Data with Model

##### In code below, use "Code for randomized α values" to simulate random α values, which will then be estimated by our model. This was used for our simulated vs recovered plots in the manuscript. In order to graph "perfect" learning rate data, use "Code for α = 1," which was plotted in our manuscript alongside participants with low, medium, and high α parameters.

In [ ]:
n_stim = 36
n_subjects = len(data['ID'].unique())

#Initial values
R = np.zeros((n_stim, n_subjects))
overall_R = np.zeros((1, n_subjects))
v_excitatory = np.zeros((n_stim, n_subjects))
v_inhibitory = np.zeros((n_stim, n_subjects))
P = np.zeros((n_stim, n_subjects))
N = np.zeros((n_stim, n_subjects))
P2 = np.zeros((n_stim, n_subjects))
N2 = np.zeros((n_stim, n_subjects))

#Code for randomized α values
gen_dist = pm.Beta.dist(2, 2, shape = n_subjects)
αD_subject_sim = gen_dist.random()
αDE_subject_sim = gen_dist.random()
αDEF_subject_sim = gen_dist.random()
αS_subject_sim = gen_dist.random()
αE_subject_sim = gen_dist.random()
αF_subject_sim = gen_dist.random()
αEF_subject_sim = gen_dist.random()
αG_subject_sim = gen_dist.random()
αH_subject_sim = gen_dist.random()
αM_subject_sim = gen_dist.random()
αN_subject_sim = gen_dist.random()
αMN_subject_sim = gen_dist.random()
αU_subject_sim = gen_dist.random()
αUM_subject_sim = gen_dist.random()
αUMN_subject_sim = gen_dist.random()


#Code for α = 1
# gen_dist = pm.Beta.dist(2, 2, shape=n_subjects)
# αD_subject_sim = np.ones(n_subjects)
# αDE_subject_sim = np.ones(n_subjects)
# αDEF_subject_sim = np.ones(n_subjects)
# αS_subject_sim = np.ones(n_subjects)
# αE_subject_sim = np.ones(n_subjects)
# αF_subject_sim = np.ones(n_subjects)
# αEF_subject_sim = np.ones(n_subjects)
# αG_subject_sim = np.ones(n_subjects)
# αH_subject_sim = np.ones(n_subjects)
# αM_subject_sim = np.ones(n_subjects)
# αN_subject_sim = np.ones(n_subjects)
# αMN_subject_sim = np.ones(n_subjects)
# αU_subject_sim = np.ones(n_subjects)
# αUM_subject_sim = np.ones(n_subjects)
# αUMN_subject_sim = np.ones(n_subjects)

#US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(36, axis=1).astype(float)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['Dcs', 'D1', 'D2', 'Ecs', 'E1', 'F', 'S', 'G', 'H', 'Mcs', 
                                                  'M1', 'N', 'Ucs', 'U1', 'U2', 'D1_abs', 'D2_abs', 'E1_abs', 
                                                  'M1_abs', 'U1_abs', 'U2_abs',
                                                  'alpha_D', 'alpha_DE', 'alpha_DEF', 'alpha_S', 'alpha_E', 'alpha_F',
                                                  'alpha_EF', 'alpha_G', 'alpha_H', 'alpha_M', 'alpha_N', 'alpha_MN',
                                                  'alpha_U', 'alpha_UM', 'alpha_UMN']].values)

stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

#Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]]).astype(float) # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

#Designate stimulus types
stimulus_type = np.ones(n_stim)
stimulus_type[[1, 4, 10, 13, 15, 17, 18, 19, 2, 14, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[1, 4, 10, 13, 15, 17, 18, 19]] = 1 #make 1st OSs = 1

OS2_type = np.zeros(n_stim)
OS2_type[[2, 14, 16, 20]] = 1 #make 2nd OSs = 1


#Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda.astype(float))
small_lambda = T.as_tensor_variable(small_lambda.astype(float))
stimuli_shown = T.as_tensor_variable(stimuli_shown)

stimuli_shown_sim = stimuli_shown.copy()
big_lambda_sim = big_lambda.copy()
small_lambda_sim = small_lambda.copy()

### Run Fake Data Simulation

In [ ]:
#Run the loop
output, updates = scan(fn=learning_function,
                    sequences=[{'input': stimuli_shown_sim[:-1, ...]},
                             {'input': big_lambda_sim},
                             {'input': small_lambda_sim}],
                    outputs_info=[v_excitatory, v_inhibitory, P, N, P2, N2],
                    non_sequences = [stimulus_type, OS1_type, OS2_type, αD_subject_sim, αDE_subject_sim, αDEF_subject_sim, αS_subject_sim, αE_subject_sim, αF_subject_sim, αEF_subject_sim, αG_subject_sim, αH_subject_sim, αM_subject_sim, αN_subject_sim, αMN_subject_sim, αU_subject_sim, αUM_subject_sim, αUMN_subject_sim])

#Get model output
V_out, Vbar_out, P_out, N_out, P2_out, N2_out = [i.eval() for i in output]

estimated_overall_R = ((V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) - (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) + \
    ((P_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) - \
    ((N_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) + \
    ((P2_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (N_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (P_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (N_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) - \
    ((N2_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (P_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (P_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (N_out * stimuli_shown_sim[1:, ...]).sum(axis=1))

overall_R_sim = estimated_overall_R.eval()

### Check parameter recovery

In [ ]:
n_subjects = len(data['ID'].unique())

#Initial values
R = np.zeros((n_stim, n_subjects))

#US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(36, axis=1).astype(float)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['Dcs', 'D1', 'D2', 'Ecs', 'E1', 'F', 'S', 'G', 'H', 'Mcs', 
                                                  'M1', 'N', 'Ucs', 'U1', 'U2', 'D1_abs', 'D2_abs', 'E1_abs', 
                                                  'M1_abs', 'U1_abs', 'U2_abs',
                                                  'alpha_D', 'alpha_DE', 'alpha_DEF', 'alpha_S', 'alpha_E', 'alpha_F',
                                                  'alpha_EF', 'alpha_G', 'alpha_H', 'alpha_M', 'alpha_N', 'alpha_MN',
                                                  'alpha_U', 'alpha_UM', 'alpha_UMN']].values)

stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

#Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]]).astype(float) # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

stimulus_type = np.ones(n_stim)
stimulus_type[[1, 4, 10, 13, 15, 17, 18, 19, 2, 14, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[1, 4, 10, 13, 15, 17, 18, 19]] = 1 #make 1st OSs = 1

OS2_type = np.zeros(n_stim)
OS2_type[[2, 14, 16, 20]] = 1 #make 2nd OSs = 1

#Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda.astype(float))
small_lambda = T.as_tensor_variable(small_lambda.astype(float))
stimuli_shown = T.as_tensor_variable(stimuli_shown)

with pm.Model() as model:
    
    # Learning rate lies between 0 and 1, so we use a beta distribution
    αD_mean = pm.Normal('αD_mean', 0.5, 10)
    αD_sd = pm.HalfCauchy('αD_sd', 10)
    
    αDE_mean = pm.Normal('αDE_mean', 0.5, 10)
    αDE_sd = pm.HalfCauchy('αDE_sd', 10)
    
    αDEF_mean = pm.Normal('αDEF_mean', 0.5, 10)
    αDEF_sd = pm.HalfCauchy('αDEF_sd', 10)
    
    αS_mean = pm.Normal('αS_mean', 0.5, 10)
    αS_sd = pm.HalfCauchy('αS_sd', 10)
    
    αE_mean = pm.Normal('αE_mean', 0.5, 10)
    αE_sd = pm.HalfCauchy('αE_sd', 10)
    
    αF_mean = pm.Normal('αF_mean', 0.5, 10)
    αF_sd = pm.HalfCauchy('αF_sd', 10)
    
    αEF_mean = pm.Normal('αEF_mean', 0.5, 10)
    αEF_sd = pm.HalfCauchy('αEF_sd', 10)
    
    αG_mean = pm.Normal('αG_mean', 0.5, 10)
    αG_sd = pm.HalfCauchy('αG_sd', 10)
    
    αH_mean = pm.Normal('αH_mean', 0.5, 10)
    αH_sd = pm.HalfCauchy('αH_sd', 10)
    
    αM_mean = pm.Normal('αM_mean', 0.5, 10)
    αM_sd = pm.HalfCauchy('αM_sd', 10)
    
    αN_mean = pm.Normal('αN_mean', 0.5, 10)
    αN_sd = pm.HalfCauchy('αN_sd', 10)
    
    αMN_mean = pm.Normal('αMN_mean', 0.5, 10)
    αMN_sd = pm.HalfCauchy('αMN_sd', 10)
    
    αU_mean = pm.Normal('αU_mean', 0.5, 10)
    αU_sd = pm.HalfCauchy('αU_sd', 10)
    
    αUM_mean = pm.Normal('αUM_mean', 0.5, 10)
    αUM_sd = pm.HalfCauchy('αUM_sd', 10)
    
    αUMN_mean = pm.Normal('αUMN_mean', 0.5, 10)
    αUMN_sd = pm.HalfCauchy('αUMN_sd', 10)

    
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1)
    αD_subject = BoundedNormal('αD', mu=αD_mean, sd=αD_sd, shape=(n_subjects,))
    αDE_subject = BoundedNormal('αDE', mu=αDE_mean, sd=αDE_sd, shape=(n_subjects,))
    αDEF_subject = BoundedNormal('αDEF', mu=αDEF_mean, sd=αDEF_sd, shape=(n_subjects,))
    αS_subject = BoundedNormal('αS', mu=αS_mean, sd=αS_sd, shape=(n_subjects,))
    αE_subject = BoundedNormal('αE', mu=αE_mean, sd=αE_sd, shape=(n_subjects,))
    αF_subject = BoundedNormal('αF', mu=αF_mean, sd=αF_sd, shape=(n_subjects,))
    αEF_subject = BoundedNormal('αEF', mu=αEF_mean, sd=αEF_sd, shape=(n_subjects,))
    αG_subject = BoundedNormal('αG', mu=αG_mean, sd=αG_sd, shape=(n_subjects,))
    αH_subject = BoundedNormal('αH', mu=αH_mean, sd=αH_sd, shape=(n_subjects,))
    αM_subject = BoundedNormal('αM', mu=αM_mean, sd=αM_sd, shape=(n_subjects,))
    αN_subject = BoundedNormal('αN', mu=αN_mean, sd=αN_sd, shape=(n_subjects,))
    αMN_subject = BoundedNormal('αMN', mu=αMN_mean, sd=αMN_sd, shape=(n_subjects,))
    αU_subject = BoundedNormal('αU', mu=αU_mean, sd=αU_sd, shape=(n_subjects,))
    αUM_subject = BoundedNormal('αUM', mu=αUM_mean, sd=αUM_sd, shape=(n_subjects,))
    αUMN_subject = BoundedNormal('αUMN', mu=αUMN_mean, sd=αUMN_sd, shape=(n_subjects,))

    
    # Run the loop
    output, updates = scan(fn=learning_function,
                      sequences=[dict(input=stimuli_shown[:-1, ...]), dict(input=big_lambda), dict(input=small_lambda)],
                      outputs_info=[v_excitatory, v_inhibitory, P, N, P2, N2],
                      non_sequences = [stimulus_type, OS1_type, OS2_type, αD_subject, αDE_subject, αDEF_subject, αS_subject, αE_subject, αF_subject, αEF_subject, αG_subject, αH_subject, αM_subject, αN_subject, αMN_subject, αU_subject, αUM_subject, αUMN_subject])
    
    # Get model output
    V, Vbar, P, N, P2, N2 = output
    
    # Calculate response - combine direct associations, 1st-order occasion setting, and 2nd-order occasion setting.
    # As a reminder: γ1 = V*Vbar, and γ2 = P*N
    R = (V - Vbar) + ((P * Vbar * V*Vbar) - (N * V *V*Vbar)) + (P2 * N * V * V*Vbar * P*N) - (N2 * P * Vbar * V*Vbar * P*N)

    # # Single R value
    estimated_overall_R = ((V * stimuli_shown[1:, ...]).sum(axis=1) - (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P2 * stimuli_shown[1:, ...]).sum(axis=1) * (N * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (P * stimuli_shown[1:, ...]).sum(axis=1) * (N * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N2 * stimuli_shown[1:, ...]).sum(axis=1) * (P * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (P * stimuli_shown[1:, ...]).sum(axis=1) * (N * stimuli_shown[1:, ...]).sum(axis=1))
    
    # This allows us to output the estimated R
    estimated_overall_R = pm.Deterministic('estimated_overall_R', estimated_overall_R)
    
    # Reshape output of the model and get categorical likelihood
    sigma = pm.HalfCauchy('sigma', 0.5)
    likelihood = pm.Normal('likelihood', mu=estimated_overall_R, sigma=sigma, observed=pd.DataFrame(overall_R_sim.squeeze()))

# Simulated Data (go to "Real Data" below if you want to skip data simulations and just get real data results)

### Fit the Model

#### Variational Inference

In [ ]:
from pymc3.variational.callbacks import CheckParametersConvergence
with model:
    approx = pm.fit(method='advi', n=40000, callbacks=[CheckParametersConvergence()])
trace = approx.sample(1000)

In [ ]:
alpha_output = pm.summary(trace, kind='stats', varnames=[i for i in model.named_vars if 'α' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i])

In [ ]:
pm.traceplot(trace, var_names=['αD_mean', 'αDE_mean', 'αDEF_mean', 'αS_mean', 'αE_mean', 'αF_mean', 'αEF_mean', 'αG_mean', 'αH_mean', 'αM_mean', 'αN_mean', 'αMN_mean', 'αU_mean', 'αUM_mean', 'αUMN_mean']);

In [ ]:
recovered_data_var = {'Simulated_αD': αD_subject_sim, 'Recovered_αD': trace['αD'].mean(axis=0), 
                      'Simulated_αDE': αDE_subject_sim, 'Recovered_αDE': trace['αDE'].mean(axis=0),
                      'Simulated_αDEF': αDEF_subject_sim, 'Recovered_αDEF': trace['αDEF'].mean(axis=0),
                      'Simulated_αS': αS_subject_sim, 'Recovered_αS': trace['αS'].mean(axis=0),
                      'Simulated_αE': αE_subject_sim, 'Recovered_αE': trace['αE'].mean(axis=0),
                      'Simulated_αF': αF_subject_sim, 'Recovered_αF': trace['αF'].mean(axis=0),
                      'Simulated_αEF': αEF_subject_sim, 'Recovered_αEF': trace['αEF'].mean(axis=0),
                      'Simulated_αG': αG_subject_sim, 'Recovered_αG': trace['αG'].mean(axis=0),
                      'Simulated_αH': αH_subject_sim, 'Recovered_αH': trace['αH'].mean(axis=0),
                      'Simulated_αM': αM_subject_sim, 'Recovered_αM': trace['αM'].mean(axis=0),
                      'Simulated_αN': αN_subject_sim, 'Recovered_αN': trace['αN'].mean(axis=0),
                      'Simulated_αMN': αMN_subject_sim, 'Recovered_αMN': trace['αMN'].mean(axis=0),
                      'Simulated_αU': αU_subject_sim, 'Recovered_αU': trace['αU'].mean(axis=0),
                      'Simulated_αUMN': αUM_subject_sim, 'Recovered_αUM': trace['αUM'].mean(axis=0),
                      'Simulated_αUMN': αUMN_subject_sim, 'Recovered_αUMN': trace['αUMN'].mean(axis=0)}

recovered_data_var = pd.DataFrame(recovered_data_var)
recovered_data_var.to_csv(os.path.join('../output/',r'2nd POS - OS2 Stimulus-Specific, Simulated vs Recovered.csv'))

In [ ]:
f, ax = plt.subplots(3, 5, sharex = True, sharey = True, figsize=(12, 7.5))
f.suptitle('Simulated vs Recovered α Parameters', y=1.02, fontsize = 16)
f.text(.5, -.02, 'Simulated α', va='center', ha='center', fontsize = 16)
f.text(-.02, .5, 'Recovered α', va='center', ha='center', fontsize = 16, rotation=90)

sns.regplot(αD_subject_sim, trace['αD'].mean(axis=0), label='αD_subject', ax=ax[0,0], color = 'black')
sns.regplot(αDE_subject_sim, trace['αDE'].mean(axis=0), label='αDE_subject', ax=ax[0,1], color = 'black')
sns.regplot(αDEF_subject_sim, trace['αDEF'].mean(axis=0), label='αDEF_subject', ax=ax[0,2], color = 'black')
sns.regplot(αS_subject_sim, trace['αS'].mean(axis=0), label='αS_subject', ax=ax[0,3], color = 'black')
sns.regplot(αE_subject_sim, trace['αE'].mean(axis=0), label='αE_subject', ax=ax[0,4], color = 'black')
sns.regplot(αF_subject_sim, trace['αF'].mean(axis=0), label='αF_subject', ax=ax[1,0], color = 'black')
sns.regplot(αEF_subject_sim, trace['αEF'].mean(axis=0), label='αEF_subject', ax=ax[1,1], color = 'black')
sns.regplot(αG_subject_sim, trace['αG'].mean(axis=0), label='αG_subject', ax=ax[1,2], color = 'black')
sns.regplot(αH_subject_sim, trace['αH'].mean(axis=0), label='αH_subject', ax=ax[1,3], color = 'black')
sns.regplot(αM_subject_sim, trace['αM'].mean(axis=0), label='αM_subject', ax=ax[1,4], color = 'black')
sns.regplot(αN_subject_sim, trace['αN'].mean(axis=0), label='αN_subject', ax=ax[2,0], color = 'black')
sns.regplot(αMN_subject_sim, trace['αMN'].mean(axis=0), label='αMN_subject', ax=ax[2,1], color = 'black')
sns.regplot(αU_subject_sim, trace['αU'].mean(axis=0), label='αU_subject', ax=ax[2,2], color = 'black')
sns.regplot(αUM_subject_sim, trace['αUM'].mean(axis=0), label='αUM_subject', ax=ax[2,3], color = 'black')
sns.regplot(αUMN_subject_sim, trace['αUMN'].mean(axis=0), label='αUMN_subject', ax=ax[2,4], color = 'black')

for i in range(3):
    for j in range(5):
        ax[0,0].set_title('α D')
        ax[0,1].set_title('α DE')
        ax[0,2].set_title('α DEF')
        ax[0,3].set_title('α S')
        ax[0,4].set_title('α E')
        ax[1,0].set_title('α F')
        ax[1,1].set_title('α EF')
        ax[1,2].set_title('α G')
        ax[1,3].set_title('α H')
        ax[1,4].set_title('α M')
        ax[2,0].set_title('α N')
        ax[2,1].set_title('α MN')
        ax[2,2].set_title('α U')
        ax[2,3].set_title('α UM')
        ax[2,4].set_title('α UMN')

plt.setp(ax, xticks=[0, .2, .4, .6, .8, 1], yticks=[0, .2, .4, .6, .8, 1])        
plt.tight_layout()

plt.savefig(os.path.join('../output/',r'2nd POS - OS2 Stimulus-Specific, Simulated vs Recovered.svg'), bbox_inches='tight')

# Real Data

### Fit the Model to Real Data

In [ ]:
n_subjects = len(data['ID'].unique())
n_stim = 36

# Initial values
R = np.zeros((n_stim, n_subjects))  # Value estimate
overall_R = np.zeros((1, n_subjects))
v_excitatory = np.zeros((n_stim, n_subjects)) 
v_inhibitory = np.zeros((n_stim, n_subjects)) 
P = np.zeros((n_stim, n_subjects))
N = np.zeros((n_stim, n_subjects))
P2 = np.zeros((n_stim, n_subjects))
N2 = np.zeros((n_stim, n_subjects))
gamma = np.ones((n_stim, n_subjects))

# US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(n_stim, axis=1)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['Dcs', 'D1', 'D2', 'Ecs', 'E1', 'F', 'S', 'G', 'H', 'Mcs', 
                                                  'M1', 'N', 'Ucs', 'U1', 'U2', 'D1_abs', 'D2_abs', 'E1_abs', 
                                                  'M1_abs', 'U1_abs', 'U2_abs',
                                                  'alpha_D', 'alpha_DE', 'alpha_DEF', 'alpha_S', 'alpha_E', 'alpha_F',
                                                  'alpha_EF', 'alpha_G', 'alpha_H', 'alpha_M', 'alpha_N', 'alpha_MN',
                                                  'alpha_U', 'alpha_UM', 'alpha_UMN']].values)
    
stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

# Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]])  # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

stimulus_type = np.ones(n_stim)
stimulus_type[[1, 4, 10, 13, 15, 17, 18, 19, 2, 14, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[1, 4, 10, 13, 15, 17, 18, 19]] = 1 #make 1st OSs = 1

OS2_type = np.zeros(n_stim)
OS2_type[[2, 14, 16, 20]] = 1 #make 2nd OSs = 1

# Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda)
small_lambda = T.as_tensor_variable(small_lambda)
stimuli_shown = T.as_tensor_variable(stimuli_shown)

with pm.Model() as model:
    
    # Learning rate lies between 0 and 1 so we use a beta distribution
    αD_mean = pm.Normal('αD_mean', 0.5, 10)
    αD_sd = pm.HalfCauchy('αD_sd', 10)
    
    αDE_mean = pm.Normal('αDE_mean', 0.5, 10)
    αDE_sd = pm.HalfCauchy('αDE_sd', 10)
    
    αDEF_mean = pm.Normal('αDEF_mean', 0.5, 10)
    αDEF_sd = pm.HalfCauchy('αDEF_sd', 10)
    
    αS_mean = pm.Normal('αS_mean', 0.5, 10)
    αS_sd = pm.HalfCauchy('αS_sd', 10)
    
    αE_mean = pm.Normal('αE_mean', 0.5, 10)
    αE_sd = pm.HalfCauchy('αE_sd', 10)
    
    αF_mean = pm.Normal('αF_mean', 0.5, 10)
    αF_sd = pm.HalfCauchy('αF_sd', 10)
    
    αEF_mean = pm.Normal('αEF_mean', 0.5, 10)
    αEF_sd = pm.HalfCauchy('αEF_sd', 10)
    
    αG_mean = pm.Normal('αG_mean', 0.5, 10)
    αG_sd = pm.HalfCauchy('αG_sd', 10)
    
    αH_mean = pm.Normal('αH_mean', 0.5, 10)
    αH_sd = pm.HalfCauchy('αH_sd', 10)
    
    αM_mean = pm.Normal('αM_mean', 0.5, 10)
    αM_sd = pm.HalfCauchy('αM_sd', 10)
    
    αN_mean = pm.Normal('αN_mean', 0.5, 10)
    αN_sd = pm.HalfCauchy('αN_sd', 10)
    
    αMN_mean = pm.Normal('αMN_mean', 0.5, 10)
    αMN_sd = pm.HalfCauchy('αMN_sd', 10)
    
    αU_mean = pm.Normal('αU_mean', 0.5, 10)
    αU_sd = pm.HalfCauchy('αU_sd', 10)
    
    αUM_mean = pm.Normal('αUM_mean', 0.5, 10)
    αUM_sd = pm.HalfCauchy('αUM_sd', 10)
    
    αUMN_mean = pm.Normal('αUMN_mean', 0.5, 10)
    αUMN_sd = pm.HalfCauchy('αUMN_sd', 10)

    
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1)
    αD_subject = BoundedNormal('αD', mu=αD_mean, sd=αD_sd, shape=(n_subjects,))
    αDE_subject = BoundedNormal('αDE', mu=αDE_mean, sd=αDE_sd, shape=(n_subjects,))
    αDEF_subject = BoundedNormal('αDEF', mu=αDEF_mean, sd=αDEF_sd, shape=(n_subjects,))
    αS_subject = BoundedNormal('αS', mu=αS_mean, sd=αS_sd, shape=(n_subjects,))
    αE_subject = BoundedNormal('αE', mu=αE_mean, sd=αE_sd, shape=(n_subjects,))
    αF_subject = BoundedNormal('αF', mu=αF_mean, sd=αF_sd, shape=(n_subjects,))
    αEF_subject = BoundedNormal('αEF', mu=αEF_mean, sd=αEF_sd, shape=(n_subjects,))
    αG_subject = BoundedNormal('αG', mu=αG_mean, sd=αG_sd, shape=(n_subjects,))
    αH_subject = BoundedNormal('αH', mu=αH_mean, sd=αH_sd, shape=(n_subjects,))
    αM_subject = BoundedNormal('αM', mu=αM_mean, sd=αM_sd, shape=(n_subjects,))
    αN_subject = BoundedNormal('αN', mu=αN_mean, sd=αN_sd, shape=(n_subjects,))
    αMN_subject = BoundedNormal('αMN', mu=αMN_mean, sd=αMN_sd, shape=(n_subjects,))
    αU_subject = BoundedNormal('αU', mu=αU_mean, sd=αU_sd, shape=(n_subjects,))
    αUM_subject = BoundedNormal('αUM', mu=αUM_mean, sd=αUM_sd, shape=(n_subjects,))
    αUMN_subject = BoundedNormal('αUMN', mu=αUMN_mean, sd=αUMN_sd, shape=(n_subjects,))
    


    
    # Run the loop
    output, updates = scan(fn=learning_function,
                      sequences=[dict(input=stimuli_shown[:-1, ...]), dict(input=big_lambda), dict(input=small_lambda)],
                      outputs_info=[v_excitatory, v_inhibitory, P, N, P2, N2],
                      non_sequences = [stimulus_type, OS1_type, OS2_type, αD_subject, αDE_subject, αDEF_subject, αS_subject, αE_subject, αF_subject, αEF_subject, αG_subject, αH_subject, αM_subject, αN_subject, αMN_subject, αU_subject, αUM_subject, αUMN_subject])
    
    # Get model output
    V, Vbar, P, N, P2, N2 = output
    
    # Calculate response - combine direct associations, 1st-order occasion setting, and 2nd-order occasion setting.
    # As a reminder: γ1 = V*Vbar, and γ2 = P*N
    R = (V - Vbar) + ((P * Vbar * V*Vbar) - (N * V *V*Vbar)) + (P2 * N * V * V*Vbar * P*N) - (N2 * P * Vbar * V*Vbar * P*N)

    # # Single R value
    estimated_overall_R = ((V * stimuli_shown[1:, ...]).sum(axis=1) - (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P2 * stimuli_shown[1:, ...]).sum(axis=1) * (N * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (P * stimuli_shown[1:, ...]).sum(axis=1) * (N * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N2 * stimuli_shown[1:, ...]).sum(axis=1) * (P * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (P * stimuli_shown[1:, ...]).sum(axis=1) * (N * stimuli_shown[1:, ...]).sum(axis=1))
       
    # This allows us to output the estimated R
    estimated_overall_R = pm.Deterministic('estimated_overall_R', estimated_overall_R)
    V = pm.Deterministic('estimated_V', V)
    Vbar = pm.Deterministic('estimated_Vbar', Vbar)
    P = pm.Deterministic('estimated_P', P)
    N = pm.Deterministic('estimated_N', N)
    P2 = pm.Deterministic('estimated_P2', P2)
    N2 = pm.Deterministic('estimated_N2', N2)
    γ1 = pm.Deterministic('estimated_γ1', V*Vbar)
    γ2 = pm.Deterministic('estimated_γ2', P*N)
          
    # Reshape output of the model and get categorical likelihood
    sigma = pm.HalfCauchy('sigma', 0.5)
    likelihood = pm.Normal('likelihood', mu=estimated_overall_R, sigma=sigma, observed=pd.DataFrame(observed_R.squeeze()))

#### Variational Inference

In [ ]:
from pymc3.variational.callbacks import CheckParametersConvergence
with model:
    approx = pm.fit(method='advi', n=40000, callbacks=[CheckParametersConvergence()])
trace = approx.sample(1000)

In [ ]:
alpha_output = pm.summary(trace, kind='stats', varnames=[i for i in model.named_vars if 'α' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i])

In [ ]:
pm.traceplot(trace, var_names=['αD_mean', 'αDE_mean', 'αDEF_mean', 'αS_mean', 'αE_mean', 'αF_mean', 'αEF_mean', 'αG_mean', 'αH_mean', 'αM_mean', 'αN_mean', 'αMN_mean', 'αU_mean', 'αUM_mean', 'αUMN_mean']);

### WAIC, R2, and Output

In [ ]:
overall_R_mean = trace['estimated_overall_R'].mean(axis=0)
overall_R_sd = trace['estimated_overall_R'].std(axis=0)

sub_ids = data['ID'].unique()

subs = [np.where(data['ID'].unique() == sub)[0][0] for sub in sub_ids]

In [ ]:
r2s = []

for n, sub in enumerate(data['ID'].unique()):
    r2s.append(r2_score(observed_R.squeeze()[~np.isnan(observed_R.squeeze()[:, n]), n], overall_R_mean[~np.isnan(observed_R.squeeze()[:, n]), n]))

group_r2 = np.median(r2s)

print(group_r2)

In [ ]:
r2s = {'R2': [r2s]}

r2s = pd.DataFrame(r2s)

r2s.to_csv(os.path.join('../output/',r'2nd POS - OS2 Stimulus-Specific, R2 Output.csv'))

In [ ]:
waic_output = pm.waic(trace)

In [ ]:
waic_output

In [ ]:
alpha_output.to_csv(os.path.join('../output/',r'2nd POS - OS2 Stimulus-Specific, Alpha Output.csv'))
waic_output.to_csv(os.path.join('../output/',r'2nd POS - OS2 Stimulus-Specific, WAIC Output.csv'))

### Plotting Real, Model-Predicted, and Perfect Learning Data

In [ ]:
data = pd.read_csv(os.path.join('../data/', "2nd POS Modeling All Data OS2.csv"))

In [ ]:
data['DV'] = ((data['DV'].values - 1) / 2) - 1

observed_R_test = data.pivot(columns = 'ID', index = 'trialseq', values = 'DV').values

##### For the following cell, you will need to have run the "Simulation Data" code near the top of this notebook using "Code for α = 1." This will allow you to re-create the three plots that are reported in the manuscript (as well as all other participants).

In [ ]:
f, ax = plt.subplots(23, 3, figsize=(36, 48), dpi = 100)

overall_R_mean = trace['estimated_overall_R'].mean(axis=0)
overall_R_sd = trace['estimated_overall_R'].std(axis=0)

sub_ids = data['ID'].unique()

subs = [np.where(data['ID'].unique() == sub)[0][0] for sub in sub_ids]
    
for n, sub in enumerate(subs):
    ax[n % 23, int(n / 23)].fill_between(range(overall_R_mean.shape[0]), overall_R_mean[:, sub] - overall_R_sd[:, sub], overall_R_mean[:, sub] + overall_R_sd[:, sub], alpha=0.3)
    ax[n % 23, int(n / 23)].plot(overall_R_mean[:, sub])
    ax[n % 23, int(n / 23)].plot(observed_R_test.squeeze()[:, sub], color='orange', linestyle='-')#participant's real data
    ax[n % 23, int(n / 23)].plot(overall_R_sim.squeeze()[:, sub], color='silver', linestyle=':', alpha = .7)#Alpha = 1; this is the correct answer if a person learned perfectly
    if n == 0:
        ax[n % 23, int(n / 23)].set_ylabel('Mean (+/-SD) overall R')
    ax[n % 23, int(n / 23)].set_ylabel('Responding (R)')
    ax[n % 23, int(n / 23)].set_xlabel('Trials')
    ax[n % 23, int(n / 23)].set_title('Sub {0}'.format(sub_ids[n]))   

plt.tight_layout()

plt.savefig(os.path.join('../output/',r'2nd POS - OS2 Stimulus-Specific, Individual Real and Estimated Responding.svg'), bbox_inches='tight')

In [ ]:
%load_ext watermark
%watermark -v -p conda,jupyterlab,numpy,pandas,theano,pymc3,sklearn,seaborn,matplotlib